### Imports
---

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import altair as alt
import pandas as pd
from src.utils.dataset import get_full_transactions_dataset

2025-02-15 14:34:22.593 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-02-15 14:34:22.595 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


### Dataframe preparation
---

In [2]:
transactions_df = get_full_transactions_dataset()

2025-02-15 14:34:22.602 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-02-15 14:34:22.603 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 14:34:23.063 
  command:

    streamlit run C:\Users\ferna\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-15 14:34:23.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 14:34:23.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 14:34:23.575 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 14:34:23.577 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 14:34:29.728 Thread 'Ma

In [3]:
account_id = '70_100428660'

In [4]:
transactions_df = transactions_df.query("sender == @account_id or receiver == @account_id")

In [5]:
transactions_df.head()

,timestamp,sender,receiver,amount_received,receiving_currency,amount_paid,payment_currency,payment_format,is_laundering
50,2022/09/01 00:25,70_100428660,10_800059F50,5105.92,US Dollar,5105.92,US Dollar,Cheque,0
99,2022/09/01 00:16,70_100428660,220_800132390,15509630.09,US Dollar,15509630.09,US Dollar,Cheque,0
100,2022/09/01 00:01,70_100428660,220_800132390,18722.79,US Dollar,18722.79,US Dollar,Credit Card,0
101,2022/09/01 00:03,70_100428660,220_800132390,19780972.52,US Dollar,19780972.52,US Dollar,Cash,0
140,2022/09/01 00:14,70_100428660,220_800190EB0,22605.52,US Dollar,22605.52,US Dollar,Cheque,0


In [6]:
transactions_df.payment_format.unique()

array(['Cheque', 'Credit Card', 'Cash'], dtype=object)

### Visualization
---

In [7]:
transactions_timeline_df = transactions_df.copy()
transactions_timeline_df["date"] = pd.to_datetime(transactions_timeline_df["timestamp"]).dt.date
daily_transactions = transactions_timeline_df.groupby('date', as_index=False)['amount_paid'].sum()
fig = px.line(
    daily_transactions,
    x="date",
    y="amount_paid",
    title='Transaction Timeline',
    labels={"date": "Date", "amount_paid": "Total Amount Paid"},
    markers=True
)
fig.update_layout(
    title_x=0.5,
    plot_bgcolor='#F9F9F9',
    paper_bgcolor="rgba(0,0,0,0)",
    font=dict(family="Arial, sans-serif", size=14, color="#004080"),
    xaxis=dict(showgrid=True, gridcolor="lightgrey"),
    yaxis=dict(showgrid=True, gridcolor="lightgrey"),
    hovermode="x unified"
)
fig.show()

In [8]:
payment_type_df = transactions_df.copy()
payment_type_df = payment_type_df.groupby("payment_format", as_index=False).amount_paid.sum()

In [9]:
custom_colors = {
    "Cheque": "#4E79A7",      
    "Credit Card": "#F28E2B", 
    "ACH": "#E15759",         
    "Cash": "#76B7B2",        
    "Wire": "#59A14F",        
    "Bitcoin": "#D4A157"
}

In [11]:
fig = px.pie(
    payment_type_df, 
    names="payment_format", 
    values="amount_paid", 
    title="Payment Method Distribution",
    hole=0.4,
    color="payment_format",
    color_discrete_map=custom_colors
)
fig.show()

In [21]:
fig = px.histogram(
    transactions_df,
    x="amount_paid",
    title="Transaction Value Distribution",
    labels={"amount_paid": "Transaction Amount"},
    color_discrete_sequence=["#F28E2B"],
    nbins=20
)

# Melhorando layout
fig.update_layout(
    xaxis_title="Transaction Amount (USD)",
    yaxis_title="Frequency",
    bargap=0.1,
    yaxis_type = 'log'
)

In [14]:
df = transactions_df.copy()
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["hour"] = df["timestamp"].dt.hour
df["weekday"] = df["timestamp"].dt.day_name()

heatmap_data = df.groupby(["weekday", "hour"], as_index=False)["amount_paid"].sum()

weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
heatmap_data["weekday"] = pd.Categorical(heatmap_data["weekday"], categories=weekday_order, ordered=True)

fig = px.density_heatmap(
    heatmap_data,
    x="hour",
    y="weekday",
    z="amount_paid",
    title="Transaction Heatmap (Hour vs. Weekday)",
    color_continuous_scale="Reds",
    nbinsx=24
)
fig.show()